In [1]:
model_name = "with_top500"

In [2]:
import numpy as np
import pandas as pd
import gc
from time import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [3]:
timesheet = [time()]
def timer(statement):
    global timesheet
    timesheet.append(time())
    print statement+" :", (timesheet[-1]-timesheet[-2]),"seconds"
timer("Init...")

Init... : 0.000192880630493 seconds


In [4]:
def load_meta(directory, prefix, modeltype):
    m_tr = pd.read_csv(directory+prefix+"_train.csv")
    m_te = pd.read_csv(directory+prefix+"_test.csv")
    data = pd.concat([m_tr, m_te], axis=0).reset_index(drop=True)
    data.columns = ["{}_{}_{}".format(c, prefix, modeltype) if c!="SK_ID_CURR" else c for c in data.columns]
    return data

def load_neptune(directory, modelname):
    m_tr = pd.read_csv(directory+"lightGBM_out_of_fold_train_predictions.csv", usecols = ["SK_ID_CURR","lightGBM_prediction"])
    m_te = pd.read_csv(directory+"lightGBM_out_of_fold_test_predictions.csv", usecols = ["SK_ID_CURR","lightGBM_prediction"])
    m_te = m_te.groupby("SK_ID_CURR")["lightGBM_prediction"].mean().reset_index()
    
    data = pd.concat([m_tr, m_te], axis=0).reset_index(drop=True)
    data.columns = ["SK_ID_CURR", "neptune_{}".format(modelname)]
    return data

def join_features(data, features):
    for item in features:
        data = data.merge(item, how = "left", on = "SK_ID_CURR")
    return data

def load_data(datafile):
    global important_columns
    colnames = [c.replace(" ","_") for c in pd.read_csv(datafile, nrows= 1).columns]
    intersection = list(set(colnames).intersection(set(important_columns)))+["SK_ID_CURR"]

    df =  pd.read_csv(datafile, names = colnames, usecols = intersection, skiprows=1).dropna(axis=1, how="all")
    timer("Loaded {} with shape {} in ".format(datafile.split("/")[-1], df.shape))
    return df

## Getting Feature Importance File

In [5]:
important_columns = pd.read_csv("../feature selector/importance/shap_importances.csv")
important_columns["feature"] = important_columns["feature"].apply(lambda x: x.replace(" ","_"))
important_columns = important_columns.loc[important_columns.shapely_mean > 0.0].reset_index(drop=True)
important_columns = important_columns.sort_values(by = "shapely_mean", ascending = False).reset_index(drop=True)
important_columns = important_columns.loc[0:500].feature.values.tolist()

## Loading Features

In [6]:
applications = load_data("../extractor/csv/application_features_V2.csv")
bureau_balance_bb = load_data("../extractor/csv/bureau_features_V1.csv")
credit_card_balance = load_data("../extractor/csv/credit_card_features_V1.csv")
installment_features = load_data("../extractor/csv/installment_features_V1.csv")
pos_cash_balance = load_data("../extractor/csv/pos_cash_features_V1.csv")
previous_apps = load_data("../extractor/csv/previous_application_features_V1.csv")

credit_card_balance_v3 = load_data("../extractor/csv/credit_card_features_V3.csv")
installment_features_v3 = load_data("../extractor/csv/installment_features_V3.csv")
pos_cash_balance_v3 = load_data("../extractor/csv/pos_cash_features_V3.csv")
previous_apps_v3 = load_data("../extractor/csv/previous_application_features_V3.csv")

base_features = [applications, bureau_balance_bb, credit_card_balance, installment_features, pos_cash_balance,previous_apps, credit_card_balance_v3,installment_features_v3,pos_cash_balance_v3, previous_apps_v3]

Loaded application_features_V2.csv with shape (356255, 75) in  : 8.69935011864 seconds
Loaded bureau_features_V1.csv with shape (305811, 84) in  : 10.8587238789 seconds
Loaded credit_card_features_V1.csv with shape (103558, 16) in  : 10.5597481728 seconds
Loaded installment_features_V1.csv with shape (339587, 100) in  : 34.4210269451 seconds
Loaded pos_cash_features_V1.csv with shape (337252, 48) in  : 21.7874219418 seconds
Loaded previous_application_features_V1.csv with shape (338857, 93) in  : 23.0144040585 seconds
Loaded credit_card_features_V3.csv with shape (103558, 6) in  : 17.0106780529 seconds
Loaded installment_features_V3.csv with shape (339587, 69) in  : 29.2455968857 seconds
Loaded pos_cash_features_V3.csv with shape (337252, 11) in  : 7.32674598694 seconds
Loaded previous_application_features_V3.csv with shape (338857, 9) in  : 8.75980710983 seconds


In [7]:
tree_meta_features = [
    load_meta("../base trees/csv/", "application", "trees"),
    load_meta("../base trees/csv/", "bureau_balance_bb", "trees"),
    load_meta("../base trees/csv/", "credit_card_balance", "trees"),
    load_meta("../base trees/csv/", "installment", "trees"),
    load_meta("../base trees/csv/", "pos_cash_balance", "trees"),
    load_meta("../base trees/csv/", "previous_apps", "trees"),
]

lr_meta_features = [
    load_meta("../base lr/csv/", "application", "lr"),
    load_meta("../base lr/csv/", "bureau_balance_bb", "lr"),
    load_meta("../base lr/csv/", "credit_card_balance", "lr"),
    load_meta("../base lr/csv/", "installment", "lr"),
    load_meta("../base lr/csv/", "pos_cash_balance", "lr"),
    load_meta("../base lr/csv/", "previous_apps", "lr"),
]

nb_meta_features = [
    load_meta("../base nb/csv/", "application", "nb"),
    load_meta("../base nb/csv/", "bureau_balance_bb", "nb"),
    load_meta("../base nb/csv/", "credit_card_balance", "nb"),
    load_meta("../base nb/csv/", "installment", "nb"),
    load_meta("../base nb/csv/", "pos_cash_balance", "nb"),
    load_meta("../base nb/csv/", "previous_apps", "nb"),
]

neptune_features = [
    load_neptune("../base neptune/m1/csv/", "m1"),
    load_neptune("../base neptune/m2/csv/", "m2"),
    load_neptune("../base neptune/m3/csv/", "m3"),
    load_neptune("../base neptune/m4/csv/", "m4"),
]

mixture_models = [
    load_meta("../base mixtures/csv/", "knn_on_selected_pca", "mixtures"),
    load_meta("../base mixtures/csv/", "lgbm_on_core_features", "mixtures"),
    load_meta("../base mixtures/csv/", "lgbm_on_gp_features", "mixtures"),
    load_meta("../base mixtures/csv/", "lr_on_core_features", "mixtures"),
    load_meta("../base mixtures/csv/", "nn", "mixtures"),
    load_meta("../base mixtures/csv/", "using_lags_bureau_data", "mixtures"),

]

l1_features = [
    load_meta("../l1/csv/", "l1_gnb", "l1"),
    load_meta("../l1/csv/", "l1_lr", "l1"),
    load_meta("../l1/csv/", "l1_tree_with_flags", "l1"),
    load_meta("../l1/csv/", "l1_tree_without_flags", "l1"),
    load_meta("../l1/csv/", "tree_with_flags_without_meta", "l1"),
    load_meta("../l1/csv/", "tree_without_flags_without_meta", "l1"),
    load_meta("../l1/csv/", "tree_on_core_features_with_meta", "l1"),
]

l2_features = [
    load_meta("../l2/csv/", "tree_on_l1_and_basemixtures_neptune_1", "l2"),
    load_meta("../l2/csv/", "tree_on_l1_and_basemixtures_neptune_2", "l2"),
]

final_models = [
    load_meta("../final_models/csv/", "my500_neptuneselected_meta", "final_models"),
    load_meta("../final_models/csv/", "my500_neptuneselected_nometa", "final_models"),
]

## Joining All Files

In [8]:
train = pd.read_csv("../data/application_train.csv", usecols = ["SK_ID_CURR","TARGET"])
test = pd.read_csv("../data/application_test.csv", usecols = ["SK_ID_CURR"])

data = pd.concat([train, test], axis=0).reset_index(drop=True)
data = join_features(data, base_features)
data = join_features(data, tree_meta_features)
data = join_features(data, lr_meta_features)
data = join_features(data, nb_meta_features)
data = join_features(data, neptune_features)
data = join_features(data, mixture_models)
data = join_features(data, l1_features)
data = join_features(data, l2_features)
data = join_features(data, final_models)

del base_features,applications, bureau_balance_bb, credit_card_balance, installment_features, pos_cash_balance,previous_apps, credit_card_balance_v3,installment_features_v3,pos_cash_balance_v3, previous_apps_v3
gc.collect()

train = data.loc[data.TARGET.notnull()].reset_index(drop=True)
test = data.loc[data.TARGET.isnull()].reset_index(drop=True)

train_id = train[["SK_ID_CURR"]]
test_id = test[["SK_ID_CURR"]]
test_id_rank = test[["SK_ID_CURR"]]
target =train.TARGET

train.drop(["SK_ID_CURR", "TARGET"], axis=1, inplace=True)
test.drop(["SK_ID_CURR","TARGET"], axis=1, inplace = True)

# Defining Model

In [9]:
import lightgbm as lgb
def model_tree(x_train, x_test, y_train, y_test, test, meta_train, meta_test,train_index, test_index,fold_id):
    dtrain = lgb.Dataset(x_train, label=y_train)
    dval = lgb.Dataset(x_test, label=y_test)
    params = {
        'num_leaves': 14, #32
        'reg_alpha': 0.04, 
        'n_jobs': -1, 
        'colsample_bytree': 0.7, 
        'silent': -1, 
        'subsample_for_bin': 200000, 
        'subsample_freq': 1, 
        'learning_rate': 0.02, # 0.02
        'boosting_type': 'gbdt', 
        'nthread': 6, 
        'min_child_weight': 40, 
        'min_split_gain': 0.0222415, 
        'n_estimators': 10000, 
        'subsample': 1, 
        'reg_lambda': 10, 
        'objective': "binary", 
        'verbose': -1, 
        'min_child_samples': 20, 
        'max_depth': 3, #8
        'class_weight': None,
        "bagging_seed" : 3143,
        "seed":1343,
        "metric":"auc"
    }
    model = lgb.train(params, dtrain, num_boost_round=5000,valid_sets=[dtrain, dval], early_stopping_rounds=200, verbose_eval=100)
    meta_train[test_index] = model.predict(x_test, num_iteration=model.best_iteration or 5000)
    meta_test.append(model.predict(test, num_iteration=model.best_iteration or 5000))
    
    global fold_roc
    fold_roc.append(roc_auc_score(y_test, meta_train[test_index]))
    # Calculate Feature Importance
    global feature_importance
    gain = model.feature_importance('gain')
    fold_feature_importance = pd.DataFrame({'feature':model.feature_name(), 'split':model.feature_importance('split'), 'gain':100 * gain / gain.sum()})
    feature_importance = feature_importance.append(fold_feature_importance, ignore_index=True) 

# Training Model

In [10]:
meta_train = np.zeros(train.shape[0])
meta_test = []
feature_importance = pd.DataFrame(columns = ["feature","split","gain"])
fold_roc = []

kf = StratifiedKFold(n_splits= 10, shuffle=True, random_state=12323)
for fold_id, (train_index, test_index) in enumerate(kf.split(train, target)):
    x_train, x_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]

    model_tree(x_train, x_test, y_train, y_test, test, meta_train, meta_test,train_index, test_index,fold_id)

test_id["TARGET"] = np.array(meta_test).T.mean(axis=1)
test_id_rank["TARGET"] = pd.DataFrame(np.array(meta_test).T).rank(pct = True).mean(axis=1)
train_id["TARGET"] = meta_train

print "Overall ROC: {},  Mean ROC: {}, STD AUC: {}".format(roc_auc_score(target, meta_train), np.mean(fold_roc), np.std(fold_roc))

/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/lightgbm/engine.py:99: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/lightgbm/basic.py:657: UserWarning: silent keyword has been found in `params` and will be ignored. Please use silent argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


Training until validation scores don't improve for 200 rounds.
[100]	training's auc: 0.800997	valid_1's auc: 0.797393
[200]	training's auc: 0.802115	valid_1's auc: 0.798101
[300]	training's auc: 0.803395	valid_1's auc: 0.798696
[400]	training's auc: 0.804531	valid_1's auc: 0.798806
[500]	training's auc: 0.80557	valid_1's auc: 0.798666
Early stopping, best iteration is:
[385]	training's auc: 0.804341	valid_1's auc: 0.798826
Training until validation scores don't improve for 200 rounds.
[100]	training's auc: 0.801522	valid_1's auc: 0.794545
[200]	training's auc: 0.802524	valid_1's auc: 0.794711
[300]	training's auc: 0.803902	valid_1's auc: 0.795065
[400]	training's auc: 0.805025	valid_1's auc: 0.795173
[500]	training's auc: 0.806132	valid_1's auc: 0.795188
[600]	training's auc: 0.8072	valid_1's auc: 0.795214
[700]	training's auc: 0.80815	valid_1's auc: 0.795188
Early stopping, best iteration is:
[597]	training's auc: 0.807156	valid_1's auc: 0.795225
Training until validation scores don't

/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

Overall ROC: 0.800362250917,  Mean ROC: 0.800555112666, STD AUC: 0.00467517600673


In [11]:
train_id.to_csv("csv/{}_train.csv".format(model_name), index=False)
test_id.to_csv("csv/{}_test.csv".format(model_name), index=False)
test_id_rank.to_csv("csv/{}_rank_test.csv".format(model_name), index=False)

In [ ]:
-